In [72]:
from prerequisite_checks import df_order as fact_order, df_shipping as fact_shipping, df_customer as dim_customer
import pandasql as ps


##### 1.	The total amount spent and the country for the Pending delivery status for each country.

In [73]:
q1 = """ SELECT country, SUM(Amount) AS total_amount_spent FROM df_order o 
LEFT JOIN dim_customer c ON o.Customer_ID = c.Customer_ID 
LEFT JOIN fact_shipping s ON o.Customer_ID = s.Customer_ID 
WHERE Status = 'Pending' GROUP BY Country """
print(ps.sqldf(q1, locals()))

  Country  total_amount_spent
0     UAE               53800
1      UK              136300
2     USA               65500


##### 2.	The total number of transactions, total quantity sold, and total amount spent for each customer, along with the product details.

In [74]:
q2 = """ WITH customer_item_agg AS (
    SELECT 
        customer_id,
        item,
        COUNT(*) AS total_transactions,
        COUNT(*) AS total_quantity_sold,
        SUM(amount) AS total_amount_spent
    FROM fact_order
    GROUP BY Customer_ID, Item
)
SELECT 
    c.customer_id,
    c.First AS first_name,
    c.Last AS last_name,
    cia.item,
    cia.total_transactions,
    cia.total_quantity_sold,
    cia.total_amount_spent
FROM customer_item_agg cia
JOIN dim_customer c ON c.Customer_ID = cia.Customer_ID
ORDER BY c.Customer_ID, cia.Item"""

print(ps.sqldf(q2, locals()))

     Customer_ID first_name   last_name      item  total_transactions  \
0              4       Eric      Carter  Mousepad                   1   
1              5    William     Jackson   DDR RAM                   1   
2              8      Jason  Montgomery   DDR RAM                   1   
3              8      Jason  Montgomery  Mousepad                   2   
4              8      Jason  Montgomery    Webcam                   1   
..           ...        ...         ...       ...                 ...   
232          247       John      Miller  Mousepad                   1   
233          249   Patricia      Garcia   DDR RAM                   1   
234          249   Patricia      Garcia   Headset                   1   
235          249   Patricia      Garcia  Mousepad                   2   
236          250    Stephen       Jones     Mouse                   1   

     total_quantity_sold  total_amount_spent  
0                      1                 200  
1                      1     

##### 3.	The maximum product purchased for each country.

In [75]:
q3 = """SELECT country, item
FROM (
  SELECT c.country, o.item, COUNT(o.customer_id) AS total_quantity,
  DENSE_RANK() OVER (PARTITION BY c.country ORDER BY COUNT(o.customer_id) DESC) AS rn
  FROM fact_order o
  JOIN dim_customer c ON o.customer_id = c.customer_id
  GROUP BY c.country, o.item
) a WHERE rn = 1"""

print(ps.sqldf(q3,locals()))

  country      item
0     UAE  Keyboard
1      UK  Mousepad
2     USA  Mousepad


##### 4.	The most purchased product based on the age category less than 30 and above 30.

In [76]:
q4 = """WITH customer_age AS (
    SELECT 
    c.customer_id,
    CASE WHEN c.age < 30 THEN '<30' ELSE '>=30' END AS age_category,
    o.item
    FROM fact_order o
    JOIN dim_customer c ON o.customer_id = c.customer_id
),
product_quantity AS (
    SELECT 
    age_category,
    item,
    COUNT(*) AS total_quantity_sold
    FROM customer_age
    GROUP BY age_category, item
),
ranked_products AS (
    SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY age_category ORDER BY total_quantity_sold DESC) AS rn
    FROM product_quantity
)
SELECT age_category, item, total_quantity_sold
FROM ranked_products
WHERE rn = 1"""

print(ps.sqldf(q4,locals()))


  age_category      item  total_quantity_sold
0          <30  Mousepad                   17
1         >=30  Keyboard                   35


##### 5.	The country that had minimum transactions and sales amount

In [77]:
q5 = """SELECT country, total_quantity
FROM (
  SELECT c.country, o.item, SUM(o.amount) AS total_quantity,
  ROW_NUMBER() OVER (PARTITION BY c.country ORDER BY SUM(o.amount)) AS rn
  FROM fact_order o
  JOIN dim_customer c ON o.customer_id = c.customer_id
  GROUP BY c.country, o.item
) a WHERE rn = 1"""

print(ps.sqldf(q5,locals()))

  country  total_quantity
0     UAE             900
1      UK            3500
2     USA            2700
